<a href="https://colab.research.google.com/github/bensonbenedict/TCM/blob/main/TCM_2_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

In [ ]:
# Define parameters for the dataset and model input
img_height, img_width = 224, 224  # Adjust to match the input size your model expects
batch_size = 32

# Dataset Path
data_dir = '/content/drive/MyDrive/tcm_dataset'

# Create an ImageDataGenerator for both training and validation, with a 20% split for validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Generator for training images
train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Binary since we have two classes: Diabetes and Healthy
    subset='training'
)

# Generator for validation images
validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/tcm_dataset'

In [ ]:
model = tf.keras.models.Sequential([
    # First convolutional layer
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Second convolutional layer
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Third convolutional layer
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Flatten the output and add fully connected layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),

    # Final output layer with a sigmoid activation for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
epochs = 50  # you can adjust the number of epochs as needed

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=epochs
)

NameError: name 'model' is not defined

In [ ]:
# Retrieve accuracy and loss values from training history
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
model.save("tongue_classification_model.h5")

In [ ]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
import io
import ipywidgets as widgets
from IPython.display import display, clear_output

# Load the saved model
model = tf.keras.models.load_model('tongue_classification_model.h5')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define image dimensions
img_height, img_width = 224, 224

def on_upload_change(change):
    clear_output(wait=True)
    display(upload_button)
    uploaded_files = change['new']
    if uploaded_files:
        img_name = list(uploaded_files.keys())[0]
        img_bytes = uploaded_files[img_name]['content']  # Extract bytes properly
        img = Image.open(io.BytesIO(img_bytes))
        print(f"Uploaded file: {img_name}")

        # Preprocess the image
        img = img.resize((img_width, img_height))
        img_array = np.array(img) / 255.0

        # Ensure 3 channels
        if len(img_array.shape) == 2:
            img_array = np.stack((img_array,) * 3, axis=-1)
        elif img_array.shape[2] == 4:
            img_array = img_array[..., :3]

        # Expand dimensions for model input
        img_array = np.expand_dims(img_array, axis=0)

        # Make prediction
        prediction = model.predict(img_array)

        # Interpret prediction
        if prediction[0][0] >= 0.5:
            result = "Diabetes"
            prob = prediction[0][0]
        else:
            result = "Healthy"
            prob = 1 - prediction[0][0]

        # Display results
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Prediction: {result}\nConfidence: {prob:.2f}")
        plt.show()
    else:
        print("No file uploaded. Please try again.")

# Create upload button
upload_button = widgets.FileUpload(accept='image/*', multiple=False)
upload_button.observe(on_upload_change, names='value')
display(upload_button)

In [ ]:
validation_filenames = validation_generator.filenames
class_indices = train_generator.class_indices  # {'Diabetes': 0, 'Healthy': 1}

In [ ]:
# Initialize lists to store file paths for each class
diabetes_files = []
healthy_files = []

# Loop through filenames and classify based on their folder
for file in validation_filenames:
    if file.startswith('Diabetes/'):
        diabetes_files.append(file)
    elif file.startswith('Healthy/'):
        healthy_files.append(file)

# Print the validation filenames for each class
print("Diabetes Validation Files:")
for file in diabetes_files:
    print(file)

print("\nHealthy Validation Files:")
for file in healthy_files:
    print(file)